# SellCollect Seller Onboarding Exercise

Hello! Welcome to my sample analysis for SellCollect a collector's marketplace<br>

**Objective of exercise:**
- Create a dashboard to improve and track the seller application process, from the first application to seller onboarding, with the aim of maximizing the number of successful onboardings while maintaining high seller quality standards.

**Dataset and exercise assumptions:**
- Contains data from application timestamps between 5/1/2022 - 7/1/2022
- The dataset needs to be analyzed quickly and the C-suite is looking for data-driven reccomendations where clarity on the dataset cannot be investigated (i.e. updating null values)
- Each application is first reviewed by the Screening Team, and then passed to the Category Team. If the application passes both screens, the applicant is moved to the 'Onboarding Status' and is invited to an onboarding session. Once the applicant has completed their onboarding, they will be live on the platform.
- Sellers are given grades for 3 categories:
    - Inventory Grade: Valuation of inventory for each seller applicant
    - Seller Grade: Seller reviews and approval rating
    - Social Grade: Social media presence (e.g., # of followers, posts relevant to what the applicant sells)
- Timestamp data definitions:
    - Application TS: when the application was submitted
    - Screening TS: when the screening team has made a decision (either accept or reject)
    - Category TS: when the category team has made a decision (either accept or reject)

The following analysis will look to showcase the following skillset:
- 🎯 **Structuring Problems:** Demonstrate how I structure my thoughts and approach problems 
- 📊 **Data Analytics Knowledge:** Ability to utilize 🐍 python and 🐼 pandas to clean and analyze data and the ability to create a Tableau dashboard
- 🔥 **Providing Data-driven Insights:** Ability to dive into the root cause and provide actionable recommendations 

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
from datetime import datetime as dt
import datetime
import time
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

### Data Cleaning
In this section, I have utlized pandas to clean the dataset along with adding additional fields. I will use in my analysis.

In [3]:
#Pulling data from csv source
df = pd.DataFrame(pd.read_csv('SellCollect_Dataset_July_2022.csv'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28760 entries, 0 to 28759
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Category                    28742 non-null  object
 1   Inbound Type                28760 non-null  object
 2   How did you hear about us?  20629 non-null  object
 3   Onboard Type                16647 non-null  object
 4   Application TS              28760 non-null  object
 5   Screening Team TS           9641 non-null   object
 6   Category Team TS            21470 non-null  object
 7   Onboarded TS                10848 non-null  object
 8   Approved - Screening Team   18660 non-null  object
 9   Rejected - Screening Team   9847 non-null   object
 10  Approved - Category Team    16617 non-null  object
 11  Rejected - Category Team    4656 non-null   object
 12  Inventory Grade             27641 non-null  object
 13  Seller Grade                27637 non-null  ob

**Application Count** 28760<br>
There are two screening processes<br>
    **Screening Team:** 9641 with Timestamp; 18660 Approved; 9847 Rejected; Not all decisions have timestamps<br>
    **Category Team:**  21470 with Timestamp; 16617 Approved; 4656 Rejected; Not all timestamps have decisions<br>
        **Onboarded:** 10848 with Timestamp (completed onboard)<br>
So I will rely on the Approved and Rejected Columns to verify success throughout the process

### Process Deviations
*Assumption: For the purpose of this analysis, I will assume the following:*<BR>
    - The Screening Team will conduct the initial screening and move applicants to "Passed initial Screen" and kickoff the category review process.<br>
        - **Applicants should not be reviewed by the category team if the applicant has not passed the initial screen** <BR>
    - The Category Team will review once the central screening team has approved the application<br>

**Finding Deviations and Data Integrity Issues**<br>
    
- Process Deviation 1: Rejected by the screening team - Category Team should not be reviewing if rejected by the central screening team

- Process Deviation 2: No screening team Review - Category Team should not be reviewing the application if rejected by the central screening team.

**Data Integry issues will not be used for data analysis and dashboard creation**
- Data Integrity 1: Double decision from Screening or Category Team. This is where the screening team and/or category team both approved and rejected the application 

- Data Integrity 2: Rejection Reason with both screen approvals




In [4]:
#Assigning Initial Statuses to Dataset
df['Status'] = np.where(df['Application TS'].notnull(), '# of Applicants', 'null')
df['Status'] = np.where(df['Approved - Screening Team'].notnull(),'Passed Initial Screen', df['Status'])

#Continue Status Building
df['Status'] = np.where(df['Approved - Category Team'].notnull(),'Onboarding', df['Status'])

#Continue Status Building
df['Status'] = np.where(df['Onboarded TS'].notnull(),'Onboarded', df['Status'])

In [5]:
#Assigning Initial Rejection statuses to data set
#Rejected during Central Screening Team
df['Status'] = np.where(df['Rejected - Screening Team'].notnull(), 'Rejected ST', df['Status'])
#Rejected by Category Team
df['Status'] = np.where(df['Rejected - Category Team'].notnull(), 'Rejected CT', df['Status'])
#Rejected during the process based on Rejection Reason
df['Status'] = np.where(((df['Rejection Reason'].notnull() & (df['Status']=="# of Applicants")) |
                        (df['Rejection Reason'].notnull() & (df['Status']=="Passed Initial Screen")))
                                                            , 'Rejected R', df['Status'])

In [6]:
#Finding Deviations and Data Integrity Issues
#Process Deviation 1: Rejected by the screening team
df['Status'] = np.where((df['Rejected - Screening Team'].notnull()) & 
                        (df['Approved - Category Team'].notnull() | 
                        df['Rejected - Category Team'].notnull())
                          , 'PD1:Rejected by Screening Team', df['Status'])
#Process Deviation 2: No screening team Review
df['Status'] = np.where((df['Rejected - Screening Team'].isnull() & df['Approved - Screening Team'].isnull()) & 
                        (df['Approved - Category Team'].notnull() | 
                        df['Rejected - Category Team'].notnull())
                          , 'PD2:No Screening Team Review', df['Status'])
#Data Integrity 1: Double decision from Screening or Category Team
df['Status'] = np.where(((df['Approved - Screening Team'] == "checked") & 
                (df['Rejected - Screening Team']=="checked")) |
                ((df['Approved - Category Team'] == "checked") & 
                (df['Rejected - Category Team']=="checked"))
                , 'DI1:Double Decision', df['Status'])
#Data Integrity 2: Rejection Reason with both screen approvals
df['Status'] = np.where(((df['Approved - Screening Team'] == "checked") & 
                (df['Approved - Category Team']=="checked")) &
                (df['Rejected - Screening Team'].isnull()) & 
                (df['Rejected - Category Team'].isnull()) &
                (df['Rejection Reason'].notnull())
                , 'DI2:Rejection Reason', df['Status'])

            

In [7]:
df.groupby(['Status'])['Application TS'].count()

Status
# of Applicants                     96
DI1:Double Decision                978
DI2:Rejection Reason                31
Onboarded                         9489
Onboarding                        5121
PD1:Rejected by Screening Team    2107
PD2:No Screening Team Review       872
Passed Initial Screen              340
Rejected CT                       2712
Rejected R                         129
Rejected ST                       6885
Name: Application TS, dtype: int64

In [8]:

df.groupby(['Status'])['Onboarded TS'].count()

Status
# of Applicants                      0
DI1:Double Decision                510
DI2:Rejection Reason                 4
Onboarded                         9489
Onboarding                           0
PD1:Rejected by Screening Team     324
PD2:No Screening Team Review       510
Passed Initial Screen                0
Rejected CT                          2
Rejected R                           0
Rejected ST                          9
Name: Onboarded TS, dtype: int64

### Quality Grade Clean Up

In [9]:
#Spliting Seller, Inventory, Social Grades to individual columns
df[['Inventory_grade','Inventory_grade_descr']] = df['Inventory Grade'].str.split(' - ',n=1,expand=True)
df[['Seller_grade', 'Seller_grade_descr']] = df['Seller Grade'].str.split(' - ', n=1, expand=True)
df[['Social_grade', 'social_grade_descr']] = df['Social Grade'].str.split(' - ', n=1,expand=True)
df.drop(['Inventory Grade', 'Seller Grade', 'Social Grade'], axis=1, inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28760 entries, 0 to 28759
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Category                    28742 non-null  object
 1   Inbound Type                28760 non-null  object
 2   How did you hear about us?  20629 non-null  object
 3   Onboard Type                16647 non-null  object
 4   Application TS              28760 non-null  object
 5   Screening Team TS           9641 non-null   object
 6   Category Team TS            21470 non-null  object
 7   Onboarded TS                10848 non-null  object
 8   Approved - Screening Team   18660 non-null  object
 9   Rejected - Screening Team   9847 non-null   object
 10  Approved - Category Team    16617 non-null  object
 11  Rejected - Category Team    4656 non-null   object
 12  Rejection Reason            9031 non-null   object
 13  Status                      28760 non-null  ob

### Timestamp Clean Up

In [11]:
#Converting the datetime columns from objects to datetime objects.
df['Application TS'] = pd.to_datetime(df['Application TS'])
df['Screening Team TS'] = pd.to_datetime(df['Screening Team TS'])
df['Category Team TS'] = pd.to_datetime(df['Category Team TS'])
df['Onboarded TS'] = pd.to_datetime(df['Onboarded TS'])
df.info()
#Normailzing Application TS to only include date and not time since other Timestamps do not include time data
df['Application TS'] = pd.to_datetime(df['Application TS']).dt.normalize()
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28760 entries, 0 to 28759
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Category                    28742 non-null  object        
 1   Inbound Type                28760 non-null  object        
 2   How did you hear about us?  20629 non-null  object        
 3   Onboard Type                16647 non-null  object        
 4   Application TS              28760 non-null  datetime64[ns]
 5   Screening Team TS           9641 non-null   datetime64[ns]
 6   Category Team TS            21470 non-null  datetime64[ns]
 7   Onboarded TS                10848 non-null  datetime64[ns]
 8   Approved - Screening Team   18660 non-null  object        
 9   Rejected - Screening Team   9847 non-null   object        
 10  Approved - Category Team    16617 non-null  object        
 11  Rejected - Category Team    4656 non-null   object    

,Category,Inbound Type,How did you hear about us?,Onboard Type,Application TS,Screening Team TS,Category Team TS,Onboarded TS,Approved - Screening Team,Rejected - Screening Team,Approved - Category Team,Rejected - Category Team,Rejection Reason,Status,Inventory_grade,Inventory_grade_descr,Seller_grade,Seller_grade_descr,Social_grade,social_grade_descr
0,Vintage Clothing,Waitlist,Word of Mouth,NaN,2022-07-01,2022-07-02,NaT,NaT,NaN,checked,NaN,NaN,Incomplete Application,Rejected ST,F,No relevant picture or video,F,No link attached,D,Link attached has little to no category relevance
1,Toys,Waitlist,Paid Ad,NaN,2022-07-01,2022-07-02,NaT,NaT,NaN,checked,NaN,NaN,Incomplete Application,Rejected ST,F,No relevant picture or video,F,No link attached,D,Link attached has little to no category relevance


In [12]:
#Finding the time delta to understand how long each process takes
df['App_to_screen'] = df['Screening Team TS'] - df['Application TS']
df['Screen_to_cat'] = df['Category Team TS'] - df['Screening Team TS']
df['Cat_to_On'] = df['Onboarded TS'] - df['Category Team TS']
df['Total_time'] = df['Onboarded TS'] - df['Application TS']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28760 entries, 0 to 28759
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   Category                    28742 non-null  object         
 1   Inbound Type                28760 non-null  object         
 2   How did you hear about us?  20629 non-null  object         
 3   Onboard Type                16647 non-null  object         
 4   Application TS              28760 non-null  datetime64[ns] 
 5   Screening Team TS           9641 non-null   datetime64[ns] 
 6   Category Team TS            21470 non-null  datetime64[ns] 
 7   Onboarded TS                10848 non-null  datetime64[ns] 
 8   Approved - Screening Team   18660 non-null  object         
 9   Rejected - Screening Team   9847 non-null   object         
 10  Approved - Category Team    16617 non-null  object         
 11  Rejected - Category Team    4656 non-null

In [13]:
#Pulling data that is apart of the non-deviated process
dirty = ['DI1:Double Decision', 'DI2:Rejection Reason', 'PD1:Rejected by Screening Team',
          'PD2:No Screening Team Review', 'PD3:Rejected & Onboarded']
dfc = df[~df['Status'].isin(dirty)]
dfc.groupby(['Status'])['Onboarded TS'].count()

Status
# of Applicants             0
Onboarded                9489
Onboarding                  0
Passed Initial Screen       0
Rejected CT                 2
Rejected R                  0
Rejected ST                 9
Name: Onboarded TS, dtype: int64

In [14]:
#Looks like there a timestamp outliers where the Category Team reviewed prior to the Screening Team
dfc.describe()

,App_to_screen,Screen_to_cat,Cat_to_On,Total_time
count,8514,6007,9477,9500
mean,0 days 12:49:53.657505285,2 days 11:49:05.563509239,7 days 10:01:15.213675213,10 days 10:51:20.084210526
std,1 days 06:07:36.625248893,4 days 03:07:11.414753040,7 days 16:19:43.806320287,9 days 09:37:00.841651718
min,0 days 00:00:00,-7 days +00:00:00,-9 days +00:00:00,0 days 00:00:00
25%,0 days 00:00:00,0 days 00:00:00,3 days 00:00:00,4 days 00:00:00
50%,0 days 00:00:00,0 days 00:00:00,5 days 00:00:00,7 days 00:00:00
75%,1 days 00:00:00,4 days 00:00:00,9 days 00:00:00,14 days 00:00:00
max,45 days 00:00:00,26 days 00:00:00,72 days 00:00:00,72 days 00:00:00


In [15]:
#5 applicants were reviewed by the Category Team prior to the Screening Team
dfc.sort_values(by='Screen_to_cat', ascending=True).head(15)

,Category,Inbound Type,How did you hear about us?,Onboard Type,Application TS,Screening Team TS,Category Team TS,Onboarded TS,Approved - Screening Team,Rejected - Screening Team,...,Inventory_grade,Inventory_grade_descr,Seller_grade,Seller_grade_descr,Social_grade,social_grade_descr,App_to_screen,Screen_to_cat,Cat_to_On,Total_time
1379,Other,Waitlist,Word of Mouth,Group,2022-06-28,2022-07-05,2022-06-28,2022-07-01,checked,NaN,...,B,Use PVC (NV: $2.5K+ product),A,"B&M, Website, OR 1K+ positive reviews w/ 90%+ ...",B,1k+ followers w/ category relevant content,7 days,-7 days,3 days,3 days
10498,Arts & Crafts,Waitlist,Word of Mouth,Group,2022-06-09,2022-06-15,2022-06-09,NaT,checked,NaN,...,F,No relevant picture or video,B,100+ positive reviews AND 90%+ Approval Rating,F,No link attached,6 days,-6 days,NaT,NaT
14123,Comics & Manga,Referral,NaN,NaN,2022-06-01,2022-06-13,2022-06-08,NaT,NaN,checked,...,F,No relevant picture or video,F,No link attached,F,No link attached,12 days,-5 days,NaT,NaT
10729,Vintage Clothing,Waitlist,Word of Mouth,Group,2022-06-08,2022-06-15,2022-06-10,2022-06-29,checked,NaN,...,B,Use PVC (NV: $2.5K+ product),F,No link attached,D,Link attached has little to no category relevance,7 days,-5 days,19 days,21 days
10148,Vintage Clothing,Waitlist,Word of Mouth,NaN,2022-06-10,2022-06-15,2022-06-12,NaT,checked,NaN,...,B,Use PVC (NV: $2.5K+ product),D,No category relevance OR <90% Approval Rating,F,No link attached,5 days,-3 days,NaT,NaT
3800,Sports,Waitlist,Word of Mouth,Group,2022-06-23,2022-06-24,2022-06-23,2022-06-30,checked,NaN,...,C,Use PVC (NV: $250+ product),F,No link attached,D,Link attached has little to no category relevance,1 days,-1 days,7 days,7 days
2,Sports,Waitlist,Paid Ad,Group,2022-07-01,2022-07-02,2022-07-02,2022-07-07,checked,NaN,...,C,Use PVC (NV: $250+ product),B,100+ positive reviews AND 90%+ Approval Rating,F,No link attached,1 days,0 days,5 days,6 days
5818,Toys,Referral,NaN,Group,2022-06-19,2022-06-20,2022-06-20,NaT,checked,NaN,...,C,Use PVC (NV: $250+ product),B,100+ positive reviews AND 90%+ Approval Rating,F,No link attached,1 days,0 days,NaT,NaT
5815,Toys,Referral,NaN,NaN,2022-06-19,2022-06-20,2022-06-20,NaT,NaN,checked,...,F,No relevant picture or video,F,No link attached,F,No link attached,1 days,0 days,NaT,NaT
5814,Estate Sales and Vintage Decor,Referral,NaN,Group,2022-06-19,2022-06-20,2022-06-20,NaT,checked,NaN,...,C,Use PVC (NV: $250+ product),A,"B&M, Website, OR 1K+ positive reviews w/ 90%+ ...",A,10K+ OR 5K+ followers w/ relevant content,1 days,0 days,NaT,NaT


In [16]:
#One applicant was Onboarded prior to Category Review
dfc.sort_values(by='Cat_to_On', ascending=True).head(1)

,Category,Inbound Type,How did you hear about us?,Onboard Type,Application TS,Screening Team TS,Category Team TS,Onboarded TS,Approved - Screening Team,Rejected - Screening Team,...,Inventory_grade,Inventory_grade_descr,Seller_grade,Seller_grade_descr,Social_grade,social_grade_descr,App_to_screen,Screen_to_cat,Cat_to_On,Total_time
2863,Trading Card Games,Waitlist,Word of Mouth,Group,2022-06-25,2022-06-25,2022-07-12,2022-07-03,checked,NaN,...,F,No relevant picture or video,C,10+ positive reviews AND 90%+ Approval Rating,F,No link attached,0 days,17 days,-9 days,8 days


In [17]:
#Converting to numbers for easier analysis in Tableau
df['App_to_screen'] = df['App_to_screen'].dt.days
df['Screen_to_cat'] = df['Screen_to_cat'].dt.days
df['Cat_to_On'] = df['Cat_to_On'].dt.days
df['Total_time'] = df['Total_time'].dt.days
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28760 entries, 0 to 28759
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Category                    28742 non-null  object        
 1   Inbound Type                28760 non-null  object        
 2   How did you hear about us?  20629 non-null  object        
 3   Onboard Type                16647 non-null  object        
 4   Application TS              28760 non-null  datetime64[ns]
 5   Screening Team TS           9641 non-null   datetime64[ns]
 6   Category Team TS            21470 non-null  datetime64[ns]
 7   Onboarded TS                10848 non-null  datetime64[ns]
 8   Approved - Screening Team   18660 non-null  object        
 9   Rejected - Screening Team   9847 non-null   object        
 10  Approved - Category Team    16617 non-null  object        
 11  Rejected - Category Team    4656 non-null   object    

In [18]:
#Assigning statuses based on time data
df['Time_Status'] = np.where(df['Screen_to_cat'] < 0 , 'PD4: Late Screen', " ")
df['Time_Status'] = np.where(df['Cat_to_On']< 0,'PD5: Late Cat', df['Time_Status'])

In [19]:
df.to_csv('Onboarding_data.csv', index=False)

# Read Me
Using the data above, I have created a Tableau dashboard that will continue the analysis. I have also created a powerpoint deck to help review the key points of the dashboard along with recommendations.